# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

Using TensorFlow backend.


# Define different types of Keras models

In [2]:
lambda_regularization = 0.20

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(units=1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_6(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

init
super init


# Create Tunable Model

In [4]:
#Shared parameters for the models
optimizer = Adam(lr=0, beta_1=0.5)
lossFunction = "mean_squared_error"
metrics = ["mse"]


#Create and compile the models
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = RULmodel_SN_6(shapeSN)
modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

nFeatures = len(selected_features)
shapeLSTM = (window_size, nFeatures)
modelRULLSTM = RULmodel_LSTM(shapeLSTM)
modelRULLSTM.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [6]:
#tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', modelRULLSTM, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_handler.data_scaler = min_max_scaler

# Load Data

In [7]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data from memory
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.63253012 -0.18639634 -0.38048616 ..., -0.66666667  0.41085271
   0.42723005]
 [-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.24031008
   0.21817178]
 [-0.31325301 -0.26095487 -0.25894666 ..., -0.33333333  0.36434109
   0.67274234]
 [-0.31325301 -0.48768258 -0.33760972 ..., -0.33333333  0.06976744
   0.2604253 ]
 [-0.30120482 -0.48506649 -0.19074949 ..., -0.33333333  0.03875969
   0.14609224]]
[ 125.  125.  125.  125.  125.]
Testing data (X, y)
[[-0.69879518 -0.24089819 -0.55536799 ..., -0.16666667  0.03875969
   0.27312897]
 [-0.09036145 -0.21037715 -0.37744767 ..., -0.5         0.03875969
   0.01518917]
 [-0.0060241  -0.22127752 -0.10634706 ...,  0.16666667  0.2248062
   0.04888152]
 [-0.25301205 -0.00414214 -0.13605672 ...,  0.16666667 -0.31782946
   0.004971  ]
 [-0.1746988   0.06256813 -0.10263336 .

In [7]:
#Load sequenced data (do not unroll sequence into a single feature vector)

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data from file
Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 125. Cros-Validation ratio 0
Printing shapes

Training data (X, y)
(17731, 30, 14)
(17731,)
Testing data (X, y)
(100, 30, 14)
(100,)
Printing first 5 elements

Training data (X, y)
[[[-0.63253012 -0.18639634 -0.38048616 ..., -0.33333333  0.42635659
    0.44932339]
  [-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.33333333
    0.46202706]
  [-0.31325301 -0.26095487 -0.25894666 ..., -0.66666667  0.25581395
    0.24275062]
  ..., 
  [-0.31325301 -0.48550251 -0.43011479 ..., -0.66666667  0.34883721
    0.07677437]
  [-0.57831325 -0.39873556 -0.36731938 ..., -0.16666667  0.2248062
    0.28555648]
  [-0.40361446 -0.01983867 -0.53308575 ..., -0.66666667  0.41085271
    0.42723005]]

 [[-0.43373494 -0.09396119 -0.29473329 ..., -0.33333333  0.33333333
    0.46202706]
  [-0.31325301 -0.26095487 -0.25894666 ..., -0.66666667  0.25581395
    0.24275062]
  [-0.31325301 -0.48768258 -0.33760972

# Train the model and test some Tunable Model functionalities

In [8]:
lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
tModel.epochs = 100
tModel.train_model(learningRate_scheduler=lrate, verbose=1)

Epoch 1/100
17731/17731 [==============================] - 3s 145us/step - loss: 8115.0706 - mean_squared_error: 8115.0706
Epoch 2/100
17731/17731 [==============================] - 1s 84us/step - loss: 7481.7579 - mean_squared_error: 7481.7579
Epoch 3/100
17731/17731 [==============================] - 1s 75us/step - loss: 7102.4167 - mean_squared_error: 7102.4167
Epoch 4/100
17731/17731 [==============================] - 1s 74us/step - loss: 6922.1051 - mean_squared_error: 6922.1051
Epoch 5/100
17731/17731 [==============================] - 1s 74us/step - loss: 6792.5539 - mean_squared_error: 6792.5539
Epoch 6/100
17731/17731 [==============================] - 1s 76us/step - loss: 6673.8719 - mean_squared_error: 6673.8719
Epoch 7/100
17731/17731 [==============================] - 1s 82us/step - loss: 6561.0284 - mean_squared_error: 6561.0284
Epoch 8/100
17731/17731 [==============================] - 1s 76us/step - loss: 6452.4893 - mean_squared_error: 6452.4893
Epoch 9/100
17731/17731

17731/17731 [==============================] - 1s 72us/step - loss: 2449.0493 - mean_squared_error: 2449.0493
Epoch 68/100
17731/17731 [==============================] - 1s 70us/step - loss: 2405.9446 - mean_squared_error: 2405.9446
Epoch 69/100
17731/17731 [==============================] - 1s 71us/step - loss: 2363.7960 - mean_squared_error: 2363.7960
Epoch 70/100
17731/17731 [==============================] - 1s 72us/step - loss: 2322.5057 - mean_squared_error: 2322.5057
Epoch 71/100
17731/17731 [==============================] - 1s 73us/step - loss: 2281.2226 - mean_squared_error: 2281.2226
Epoch 72/100
17731/17731 [==============================] - 1s 73us/step - loss: 2240.8669 - mean_squared_error: 2240.8669
Epoch 73/100
17731/17731 [==============================] - 1s 72us/step - loss: 2202.2826 - mean_squared_error: 2202.2826
Epoch 74/100
17731/17731 [==============================] - 1s 77us/step - loss: 2162.4500 - mean_squared_error: 2162.4500
Epoch 75/100
17731/17731 [===

In [9]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 1ms/step
scores
Engine 1, Predicted RUL [ 68.16468811], Rounded RUL 68.0, Real RUL 112.0
Engine 2, Predicted RUL [ 68.16400909], Rounded RUL 68.0, Real RUL 98.0
Engine 3, Predicted RUL [ 44.10900497], Rounded RUL 44.0, Real RUL 69.0
Engine 4, Predicted RUL [ 68.10655212], Rounded RUL 68.0, Real RUL 82.0
Engine 5, Predicted RUL [ 68.15187073], Rounded RUL 68.0, Real RUL 91.0
Engine 6, Predicted RUL [ 68.15992737], Rounded RUL 68.0, Real RUL 93.0
Engine 7, Predicted RUL [ 68.16461182], Rounded RUL 68.0, Real RUL 91.0
Engine 8, Predicted RUL [ 68.12329102], Rounded RUL 68.0, Real RUL 95.0
Engine 9, Predicted RUL [ 68.16430664], Rounded RUL 68.0, Real RUL 111.0
Engine 10, Predicted RUL [ 68.03411865], Rounded RUL 68.0, Real RUL 96.0
Engine 11, Predicted RUL [ 68.07137299], Rounded RUL 68.0, Real RUL 97.0
Engine 12, Predicted RUL [ 68.16177368], Rounded RUL 68.0, Real RUL 124.0
Engine 13, Predicted RUL [ 68.07537842], Rounded RUL 68.0, Real RUL 